# Train/Eval split by month (April?June train)
Loading the full dataset and creating train/eval pandas DataFrames.

In [ ]:
import pandas as pd
from pathlib import Path

data_path = Path('Data/bank-additional-full.csv')
df = pd.read_csv(data_path, sep=';')
df.head()


In [ ]:
train_months = {'apr', 'may', 'jun'}
mask = df['month'].str.lower().isin(train_months)
train_df = df[mask].copy()
eval_df = df[~mask].copy()

print(f'Train rows: {len(train_df)}  Eval rows: {len(eval_df)}')
print('Train y counts:
', train_df['y'].value_counts())
print('Eval y counts:
', eval_df['y'].value_counts())


In [ ]:
# Month breakdowns to verify split boundary
print('Train months:')
display(train_df['month'].value_counts())
print('Eval months:')
display(eval_df['month'].value_counts())


## Model: predict high vs low response
Using logistic regression with one-hot encoding to classify clients as high-response (`yes`) or low-response (`no`).


In [ ]:
# Prepare features/target
target_map = {'yes': 1, 'no': 0}
y_train = train_df['y'].map(target_map)
y_eval = eval_df['y'].map(target_map)

X_train = train_df.drop(columns=['y'])
X_eval = eval_df.drop(columns=['y'])

cat_cols = [c for c in X_train.columns if X_train[c].dtype == 'object']
num_cols = [c for c in X_train.columns if X_train[c].dtype != 'object']
print(f"Categorical cols: {len(cat_cols)}  Numeric cols: {len(num_cols)}")


In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

preprocess = ColumnTransformer(
    transformers=[
        ('categorical', OneHotEncoder(handle_unknown='ignore'), cat_cols),
        ('numeric', 'passthrough', num_cols),
    ],
    remainder='drop'
)

clf = Pipeline(steps=[
    ('preprocess', preprocess),
    ('model', LogisticRegression(max_iter=500, class_weight='balanced', n_jobs=-1)),
])
clf


In [ ]:
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix

clf.fit(X_train, y_train)

eval_pred = clf.predict(X_eval)
eval_proba = clf.predict_proba(X_eval)[:, 1]

print(f'Eval ROC-AUC: {roc_auc_score(y_eval, eval_proba):.3f}')
print('Classification report (eval):')
print(classification_report(y_eval, eval_pred, target_names=['low-response', 'high-response']))

cm = pd.DataFrame(
    confusion_matrix(y_eval, eval_pred),
    index=['true low', 'true high'],
    columns=['pred low', 'pred high'],
)
cm


In [ ]:
# Score the eval set to prioritize calls
scored_eval = eval_df.copy()
scored_eval['high_response_score'] = eval_proba
scored_eval[['high_response_score', 'y']].describe()


In [ ]:
# Top 5 likely responders
scored_eval.sort_values('high_response_score', ascending=False)[['high_response_score', 'y', 'job', 'education', 'contact']].head()
